In [ ]:
import mcstasscript as ms
import mcstasscript.jb_interface as ms_widget
import scipp as sc
import plopp as pp
import numpy as np

import matplotlib.pyplot as plt

# %matplotlib widget

import make_SANS_instrument

In [ ]:
my_configurator = ms.Configurator()
my_configurator.set_mcrun_path("/usr/bin/")
my_configurator.set_mcstas_path("/usr/share/mcstas/3.3/")

# SANS exercise


In this notebook you will get this simplified SANS instrument and answer a few questions about the results. You will also get to improve it and run experiments with both with and without a sample.

### Get an instrument object
Here from a function, in future could be online repository

In [ ]:
instrument = make_SANS_instrument.make(input_path="run_folder")
nexus_mode = False

## Investigate instrument
First investigate the instrument object *instrument* using some of the available methods. All the methods that help do that start with the word show. In particular, look at what parameters are available and take a look at the instrument geometry.

In [ ]:
instrument.show_parameters()

In [ ]:
instrument.show_instrument()

## Set parameters
Before running the instrument we need to set some parameters. The most important one is the *detector_distance* parameter describing the distance between the sample and the detector. Given the need for high angular precision in determining the scattering angle of the neutron, which of these would be best?

- A: 1 m
- B: 2 m
- C: 3 m

Could instead have question on wavelength band to test time of flight knowledge.

In [ ]:
# quiz C

Set the parameters of the instrument using the *set_parameters* method.
- sample_distance: 150 m
- wavelength: 6 Å
- wavelength bad: 1.5 Å
- enable_sample: 0
- n_pulses: 1

In [ ]:
instrument.set_parameters(
    sample_distance=150, wavelength=6, d_wavelength=1.5, enable_sample=0, n_pulses=1
)

## Run the instrument
Now the simulation can be executed with the backengine method. Store the returned data in a python variable called data.

In [ ]:
ncpu = 2
instrument.settings(ncount=5e6, mpi=ncpu, suppress_output=True)

In [ ]:
data = instrument.backengine()
data

In [ ]:
# ms_widget.show(data)

to_plot = ["signal", "signal_tof", "signal_tof_all"]

ms.make_sub_plot(
    [ms.name_search(key, data) for key in to_plot],
    figsize=(12, 8),
    log=True,
    orders_of_mag=5,
)

### Interpretation of the data
The detector is a He3 tube centered 25 cm above the beam height and with a metal casing. 

What does the signal look like without sample?
- A: Most of the signal close to the direct beam
- B: Flat signal over detector height 
- C: Most of the signal is far away from the direct beam

In [ ]:
# quiz A

Is this a problem for a SANS instrument?
- A: Yes
- B: No

In [ ]:
# quiz A

How can it be improved?
- A: By adding a Velocity selector
- B: By adding a Chopper
- C: By adding a Beamstop
- D: By adding a Slit

In [ ]:
# quiz C

## Improve the instrument
In order to improve the performance of the instrument, we will add a McStas component. The first aspect to consider when doing so is where to place it, both in the component sequence and its physical location. We start by looking at the code sequence.

### McStas sequence 
Use either the *show_diagram* or *show_components* method on the instrument object to get an overview of the component sequence in the instrument. Where would you place the new component?

- A: After the source
- B: Before the sample position
- C: After the sample position
- D: Before the detector position

In [ ]:
instrument.show_diagram()

In [ ]:
# quiz C and D correct

### Which component
Now we need to select what type of component to add to the instrument, here we will need the *Beamstop* component. Use the *component_help* method on the instrument to learn more about this component.

In [ ]:
instrument.component_help("Beamstop")

### Add chopper component and set parameters
Use the *add_component* method on the instrument to add a chopper. Place it in the component sequence by using either the *before* or *after* keyword argument.

Set the parameters:
 - yheight: 0.05 m
 - radius: 0.7 m
 - nslit: 1.0
 - nu, delay and theta_0: To the variables calculated in the instrument (use quotation marks)

In [ ]:
beamstop = instrument.add_component("beamstop", "Beamstop", before="detector_position")
beamstop.set_parameters(xwidth=0.1, yheight=0.02)

In [ ]:
# quiz(instrument)

### Placing the component in space
The next physical location of the component need to be specified, which is done using the *set_AT* component. This method takes a list of 3 numbers, corresponding to the *x*, *y* and *z* coordinates of the component. One can also specify in what coordinate system one wants to work, which can be that of any preceeding component. Use the *RELATIVE* keyword to work in the *source* coordinate system. The position of the chopper is needed for calculating phase, so it is available as a variable in the instrument, use this variable to set the position.

In [ ]:
beamstop.set_AT([0, 0, "0.9*detector_distance"], RELATIVE="sample_position")

In [ ]:
# quiz(instrument)

### Verify new component
Now that the chopper has been added to the instrument, lets show the component sequence again to verify it was added correctly.

In [ ]:
instrument.show_diagram()

## Run improved instrument
Run the improved instrument with the following parameters:
- sample_distance: 150 m
- d_wavelength: 1.5 Å
- enable_sample: 0
- n_pulses: 1

In [ ]:
instrument.settings(ncount=5e6, mpi=ncpu, suppress_output=True)

if nexus_mode:
    instrument.set_parameters(
        sample_distance=150, d_wavelength=1.5, enable_sample=0, n_pulses=3, integration_time=5E4
    )
    instrument.settings(
        output_path="SANS_without_sample_NeXus_3_pulse", NeXus=nexus_mode, ncount=1.5e7
    )
    background_data = instrument.backengine()

In [ ]:
instrument.set_parameters(
    sample_distance=150, d_wavelength=1.5, enable_sample=0, n_pulses=1, integration_time=5E4
)
instrument.settings(
    output_path="SANS_without_sample_NeXus_1_pulse", NeXus=nexus_mode, ncount=5e6
)
background_data = instrument.backengine()

In [ ]:
background_data

In [ ]:
ms.make_sub_plot(
    [ms.name_search(key, background_data) for key in to_plot],
    figsize=(12, 8),
    log=True,
    orders_of_mag=5,
)
# ms_widget.show(data)
# ms.make_sub_plot([improved_data[0], improved_data[3]], figsize=(12, 8), log=True, orders_of_mag=10)

Do you see an improvement compared to earlier results?
- A: Yes
- B: No

In [ ]:
# quiz A

## Run with sample
Now the sample can be added by setting the *enable_sample* parameter to one and calling the *backengine* method again.

In [ ]:
if nexus_mode:
    instrument.settings(
        ncount=1.5e7,
        mpi=ncpu,
        suppress_output=True,
        output_path="SANS_with_sample_NeXus_3_pulse",
        NeXus=nexus_mode,
    )
    instrument.set_parameters(
        sample_distance=150, d_wavelength=1.5, enable_sample=1, n_pulses=3, integration_time=500
    )

    sample_data = instrument.backengine()

In [ ]:
instrument.settings(
    ncount=5e6,
    mpi=ncpu,
    suppress_output=True,
    output_path="SANS_with_sample_NeXus_1_pulse",
    NeXus=nexus_mode,
)
instrument.set_parameters(
    sample_distance=150, d_wavelength=1.5, enable_sample=1, n_pulses=1, integration_time=500
)

sample_data = instrument.backengine()

In [ ]:
ms.make_sub_plot(
    [ms.name_search(key, sample_data) for key in to_plot],
    figsize=(12, 8),
    log=True,
    orders_of_mag=10,
)

Question to check the students are seeing the final dataset.